# Optimal Hedged Monte Carlo

Author: Jerry Xia

Date: 2018/06/19

## 1 Introduction

### 1.1 Facts
* The option price is not simply the average value of the discounted future pay-off over the objective (or historical) probability distribution
* The requirement of absence of arbitrage opportunities is equivalent to the existence of "risk-neutral measure", such that the price is indeed its average discounted future pay-off.
* Risk in option trading cannot be eliminated

### 1.2 Objective
* It would be satisfactory to have a option theory where the objective stochastic process of the underlying is used to calculated the option price, the hedge strategy and the *residual risk*.

### 1.3 Advantages
* It is a versatile methods to price complicated path-dependent options.
* Considerable variance reduction scheme for Monte Carlo
* It provide not only a numerical estimate of the option price, but also of the optimal hedge strategy and of the residual risk.
* This method does not rely on the notion of risk-neutral measure, and can be used to any model of the true dynamics of the underlying

## 2 Underlying dynamics

### Black-Scholes Model
$$dS = r S dt + \sigma S dW_t$$
$$log S_{t+1} = log S_t +(r - \frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t} \epsilon$$
where
    $$\epsilon \sim N(0,1)$$


## 3 Methodology

### Simbols Definition
Option price always requires to work backward. That is because the option price is known exactly at the maturity. As with other schemes, we determine the option price step by step from the maturity $t=K\tau=T$ to the present time $t=0$. The unit of time being $\tau$, for example, one day. We simulate $N$ trajectories. In trajectory i, the price of the underlying asset at time $k\tau$ is denoted as $S_k^{(i)}$. The price of the derivative at time $k\tau$ is denoted as $C_k$, and the hedge function is $H_k$. We define an optimal hedged portfolio as
$$W_k^{(i)} = C_k(S_k^{(i)}) + H_k(S_k^{(i)})S_k^{(i)}$$
The one-step change of our portfolio is
$$\Delta W_k^{(i)}= df(k,k+1) C_{k+1}(S_{k+1}^{(i)}) - C_k(S_k^{(i)}) + H_k(S_{k}^{(i)}) (df(k,k+1) S_{k+1}^{(i)} - S_{k}^{(i)})$$
Where $df(k,k+1)$ is the discounted factor from time $k\tau$ to $(k+1) \tau$

### Objective
The optimal hedged algorithm can be interpreted as the following optimal problem
\begin{align}
\mbox{minimize}\quad & \quad Var[\Delta W_k]\\
\mbox{subject to}\quad & \quad E[\Delta W_k]=0
\end{align}

### Basis Functions
The original optimization is very difficult to solve. Thus we assume a set of basis function and solved it in such subspace.
$$
C_k(\cdot) = \sum_{i=0}^{N_C} a_{k,i} A_i(\cdot)\\
H_k(\cdot) = \sum_{i=0}^{N_H} b_{k,i} B_i(\cdot)
$$
The basis functions $A_i$ and $B_i$ are priori determined and need not to be identical. The coefficients $a_i$ and $b_i$ can be calibrated by solving the optimal problem.

### Numerical Solution
\begin{align}
\mbox{minimize}\quad & \quad \frac{1}{N} \sum_{i=1}^N \Delta W_k^{(i)2}\\
\mbox{subject to}\quad & \quad \frac{1}{N} \sum_{i=1}^N \Delta W_k^{(i)}=0
\end{align}

Denote the discounted forward underlying price change at time $k\tau$ as

$$\Delta S_k = df(k,k+1) S_{k+1} - S_k$$

Define

\begin{align}
Q_k &= \begin{bmatrix}
    -A_{k,1}(S_k^{(1)}) & \cdots & -A_{k,N_C}(S_k^{(1)}) & B_{k,1}(S_k^{(1)})\Delta S_k^{(1)}& \cdots  & B_{k,N_H}(S_k^{(1)})\Delta S_k^{(1)} \\
    -A_{k,1}(S_k^{(2)}) & \cdots & -A_{k,N_C}(S_k^{(2)}) & B_{k,1}(S_k^{(2)})\Delta S_k^{(2)}& \cdots  & B_{k,N_H}(S_k^{(1)})\Delta S_k^{(2)} \\
    \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
    -A_{k,1}(S_k^{(N)}) & \cdots & -A_{k,N_C}(S_k^{(N)}) & B_{k,1}(S_k^{(N)})\Delta S_k^{(N)}& \cdots  & B_{k,N_H}(S_k^{(N)})\Delta S_k^{(N)}
    \end{bmatrix}\\\\
c_k &= (a_{k,1}, \cdots a_{k,N_C}, b_{k,1}, \cdots, b_{k,N_H})^T\\\\
v_{k} &= df(k,k+1) C_{k+1}(S_{k+1}^{})
\end{align}

As for $v_k$, note that we know the exact value at maturity, which means there is no need to approximate price in terms of basis functions, that is

\begin{align}
v_k = \begin{cases}
df(N-1,N)\ payoff(S_N),\quad & k=N-1\\
df(k,k+1)\ \sum_{i=1}^{N_C} A_i(S_{k+1}), \quad & k<N-1
\end{cases}
\end{align}

Then, the optimization problem can be expressed as

\begin{align}
\arg\min_{c_k}\quad & \quad (v_{k} + Q_k c_k)^T (v_{k} + Q_k c_k)\\
\mbox{subject to}\quad & \quad 1_{[N\times1]}^T (v_{k}  + Q_k c_k)=0
\end{align}

In step k, since we already know the information ($v_{k}$) in step k+1. By canceling the constant term, the optimal problem can be simplified as the following 

\begin{align}
\arg\min_{c_k}\quad & \quad 2 v_{k}^T Q_k c_k + c_k^T Q_k^T Q_k c_k\\
\mbox{subject to}\quad & \quad 1_{[N\times1]}^T v_{k}  + 1_{[N\times1]}^T Q_k c_k=0
\end{align}

In [1]:
import numpy as np
import scipy as sp

In [44]:
risk_free_rate = 0.02
time_to_maturity = 1
volatility = 0.3
strike = 100
stock_price = 100
n_trails = 1000
n_steps = 200
func_list = [lambda x: x**0, lambda x: x, lambda x: x**2]

In [34]:
class MonteCarlo:
    def __init__(self,S0,K,T,r,sigma,underlying_process="geometric brownian motion"):
        self.underlying_process = underlying_process
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        
    def simulate(self, n_trails, n_steps):
        dt = self.T/n_steps
        if(self.underlying_process=="geometric brownian motion"):
#             first_step_prices = np.ones((n_trails,1))*np.log(self.S0)
            log_price_matrix = np.zeros((n_trails,n_steps))
            normal_matrix = np.random.normal(size=(n_trails,n_steps))
            cumsum_normal_matrix = normal_matrix.cumsum(axis=1)
#             log_price_matrix = np.concatenate((first_step_prices,log_price_matrix),axis=1)
            deviation_matrix = cumsum_normal_matrix*self.sigma*np.sqrt(dt) + \
    (self.r-self.sigma**2/2)*dt*np.arange(1,n_steps+1)
            log_price_matrix = deviation_matrix+np.log(self.S0)
            price_matrix = np.exp(log_price_matrix)
        return price_matrix

In [35]:
mc = MonteCarlo(stock_price,strike,time_to_maturity,risk_free_rate,volatility)

In [36]:
price_matrix = mc.simulate(n_trails,n_steps)

In [37]:
price_matrix.shape

(1000, 200)

In [11]:
funcs = [lambda x: x**0, lambda x: x, lambda x: x**2]
np.array([func(np.array([3,2,4,5,6])) for func in funcs]).T

array([[ 1,  3,  9],
       [ 1,  2,  4],
       [ 1,  4, 16],
       [ 1,  5, 25],
       [ 1,  6, 36]])

In [12]:
A*np.array([1,2,3,4,5,6,7])

array([[ 2,  4,  0, 12, 10, 18, 28],
       [ 3,  4, 12,  4,  0,  0,  0],
       [ 1,  4,  0, 16, 10,  0,  7],
       [ 2,  4, 12, 12, 10,  0, 14],
       [ 3,  4, 12, 16,  0, 18, 21],
       [ 2,  4,  6,  0,  0, 12, 14],
       [ 4,  4,  9,  8, 20, 18, 14]])

In [43]:
def calculate_Q_matrix(price_matrix,r,dt,step_k,func_list=[lambda x: x**0, lambda x: x, lambda x: x**2]):
    n_trails,n_steps = price_matrix.shape
    S_k = price_matrix[:,step_k-1]
    S_kp1 = price_matrix[:,step_k]
    DS = np.exp(-r*dt)*S_kp1 - S_k
    A = np.array([func(S_k) for func in func_list]).T
    B = (np.array([func(S_k) for func in func_list])*DS).T
    return np.concatenate((-A,B),axis=1)

In [39]:
calculate_Q_matrix(price_matrix,r=risk_free_rate,dt=time_to_maturity/n_steps,step_k=n_steps-1)

array([[-1.00000000e+00, -5.80804380e+01, -3.37333727e+03,
         1.17653916e+00,  6.83339098e+01,  3.96886341e+03],
       [-1.00000000e+00, -1.48596220e+02, -2.20808365e+04,
         5.35132534e-01,  7.95186715e+01,  1.18161740e+04],
       [-1.00000000e+00, -7.68211527e+01, -5.90148950e+03,
         1.36866402e+00,  1.05142348e+02,  8.07715635e+03],
       ...,
       [-1.00000000e+00, -1.43430939e+02, -2.05724343e+04,
        -3.19022186e+00, -4.57576518e+02, -6.56306296e+04],
       [-1.00000000e+00, -8.13940345e+01, -6.62498886e+03,
         7.80779083e-01,  6.35507597e+01,  5.17265273e+03],
       [-1.00000000e+00, -1.48125458e+02, -2.19411514e+04,
        -3.63555864e+00, -5.38518789e+02, -7.97683425e+04]])

In [40]:
def discounted_function(risk_free_rate,interval):
    return np.exp(-risk_free_rate*interval)

In [42]:
def calculate_v_vector(df,Ckp1):
    return df*Ckp1

In [45]:
def calculate_C_vector(a,S,func_list):
    return np.dot(a,[func(S) for func in func_list])

In [49]:
for i in range(10,0,-1):
    print(i)

10
9
8
7
6
5
4
3
2
1


In [52]:
def pricing():
    n_trails, n_steps = price_matrix.shape
    vkp1 = (price_matrix[:,-1] - strike)
    vkp1 = np.where(vK<0,0,vK)
    for k in range(n_steps-2,-1,-1):
        Sk = price_matrix[:,k]
        Skp1 = price_matrix[:,k+1]
        Q = calculate_Q_matrix(price_matrix,r=risk_free_rate,dt=time_to_maturity/n_steps,step_k=k)
        
    
        

In [53]:
pricing()

[  0.          49.14626612   0.           0.           7.31607813
   0.           0.          61.99645926  12.03191711  58.60519694
   0.           0.           0.          14.90590267   0.
   9.16992825   1.73352783   0.           0.           7.45175918
  10.74008574   0.           0.           0.           0.
   6.33959673  98.40745525   0.           0.           9.60220027
   0.           0.           0.           0.          42.30998994
   0.          68.43515354  15.38252755  53.82785131   0.
   0.          15.35883087  33.22080621   0.          42.04204198
  54.956158    32.53899055  43.63776388  94.12236999   0.
  71.9642412   15.46550234  30.81350107  24.10043508   0.
  65.45463724  78.36718272   2.70587914   0.           6.04082834
  13.81076032   0.           0.           0.          14.95247999
   0.           0.           0.          73.39270148   0.
   0.           9.77538545   0.          31.36128546   0.
   8.20064     61.05167911   0.           0.           0.
  36.821